In [54]:
import MetaTrader5 as mt5
import pandas as pd
import sys
import pytz
from datetime import datetime
from time import gmtime, strftime

def initialize():

    login = 52029003
    password = '0a6Dx9o5!HnsaC'
    server = 'ICMarketsEU-Demo'
    
    # get current date and time
    if not mt5.initialize("C:/Program Files/FTMO MetaTrader 5 - Copy/terminal64.exe"):
        print("initialize() failed, error code =", mt5.last_error())
        quit()
    else:
        print("initialize() success")
        tz_kyjev = pytz.timezone('Europe/Kiev')
        datetime_kyjev = datetime.now(tz_kyjev)
        print("Time: ",datetime_kyjev.strftime("%H:%M:%S"))

    mt5.login(login, password, server)

    acc_info = mt5.account_info()
    if acc_info is None:
        print("Failed to get account info")

    login_number = acc_info.login
    print("Login number: ", login_number)
        # establish connection to the MetaTrader 5 terminal

initialize()

bars = mt5.copy_rates_from_pos("UK100", mt5.TIMEFRAME_M5, 0, 99999)
df = pd.DataFrame(bars)  
df['time'] = pd.to_datetime(df['time'], unit='s')
df

initialize() success
Time:  23:13:10
Login number:  52029003


,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-07-17 18:45:00,7410.2,7410.7,7409.2,7410.2,30,200,0
1,2023-07-17 18:50:00,7410.2,7410.7,7409.7,7410.2,33,200,0
2,2023-07-17 18:55:00,7410.2,7411.2,7409.7,7411.2,24,200,0
3,2023-07-17 19:00:00,7411.2,7411.2,7410.7,7411.2,21,200,0
4,2023-07-17 19:05:00,7411.2,7412.7,7411.2,7412.7,6,200,0
...,...,...,...,...,...,...,...,...
99994,2024-12-23 22:50:00,8120.8,8133.8,8120.8,8132.8,216,100,0
99995,2024-12-23 22:55:00,8132.5,8136.0,8124.5,8126.0,169,100,0
99996,2024-12-23 23:00:00,8126.3,8141.8,8126.1,8130.5,871,500,0
99997,2024-12-23 23:05:00,8130.7,8135.9,8130.3,8135.5,451,500,0


In [55]:
def split_by_next_row_three_ways(df):
    """
    Split DataFrame into three based on next row's open/close relationship
    
    Parameters:
        df (pandas.DataFrame): DataFrame with 'open' and 'close' columns
    
    Returns:
        tuple: (price_increased, price_equal, price_decreased)
    """
    # Create conditions for each case
    increased_condition = df['open'] < df['close']
    equal_condition = df['open'] == df['close']
    
    # Shift conditions UP by 1 (looking at next row's conditions)
    next_increased = increased_condition.shift(-1)
    next_equal = equal_condition.shift(-1)
    
    # Fill NaN values with False
    next_increased = next_increased.fillna(False)
    next_equal = next_equal.fillna(False)

    
    
    # Creating the three separate DataFrames
    price_increased = df[next_increased]
    price_equal = df[next_equal]
    price_decreased = df[~(next_increased | next_equal)]  # Everything else
    
    return price_increased, price_equal, price_decreased

increased, equal, decreased = split_by_next_row_three_ways(df)
increased = increased.copy()
equal = equal.copy()
decreased = decreased.copy()    

print("Rows where next row has open < close:")
print(increased)
print("\nRows where next row has open == close:")
print(equal)
print("\nRows where next row has open > close:")
print(decreased)

Rows where next row has open < close:
                     time    open    high     low   close  tick_volume  \
1     2023-07-17 18:50:00  7410.2  7410.7  7409.7  7410.2           33   
3     2023-07-17 19:00:00  7411.2  7411.2  7410.7  7411.2           21   
4     2023-07-17 19:05:00  7411.2  7412.7  7411.2  7412.7            6   
6     2023-07-17 19:15:00  7416.5  7416.5  7414.7  7414.7           18   
7     2023-07-17 19:20:00  7414.2  7415.7  7413.7  7415.7           15   
...                   ...     ...     ...     ...     ...          ...   
99992 2024-12-23 22:40:00  8118.5  8121.0  8117.3  8118.0          181   
99993 2024-12-23 22:45:00  8118.3  8121.8  8118.3  8120.5          214   
99995 2024-12-23 22:55:00  8132.5  8136.0  8124.5  8126.0          169   
99996 2024-12-23 23:00:00  8126.3  8141.8  8126.1  8130.5          871   
99997 2024-12-23 23:05:00  8130.7  8135.9  8130.3  8135.5          451   

       spread  real_volume  
1         200            0  
3         200  

In [68]:
def get_candle_by_body(df):
    # Calculate absolute body percentage for each candle
    body_pct = abs(df['open'] - df['close']) / (df['high'] - df['low'])
    
    # Calculate if candle is bullish (close > open)
    is_bullish = df['close'] > df['open']
    
    # For each row, check if previous candle had >80% body
    prev_big_body = (body_pct.shift(1) >= 0.80).fillna(False)
    
    # Check previous candle direction
    prev_bullish = is_bullish.shift(1).fillna(False)
    
    # Return filtered DataFrames split by previous candle direction
    bullish_df = df[prev_big_body & prev_bullish]  # Previous candle was bullish
    bearish_df = df[prev_big_body & ~prev_bullish]  # Previous candle was bearish
    
    return bullish_df, bearish_df

prevBearish, prevBullish  = get_candle_by_body(df)

In [79]:
prevBullish[(prevBullish.open - prevBullish.low) >= 1]

,time,open,high,low,close,tick_volume,spread,real_volume
73,2023-07-18 01:50:00,7413.3,7413.5,7412.2,7413.3,23,500,0
86,2023-07-18 02:55:00,7413.0,7413.1,7411.6,7411.6,60,500,0
87,2023-07-18 03:00:00,7411.7,7413.9,7406.1,7413.4,47,400,0
105,2023-07-18 04:30:00,7418.4,7418.4,7414.6,7415.4,32,400,0
107,2023-07-18 04:40:00,7409.1,7409.1,7403.9,7403.9,57,400,0
...,...,...,...,...,...,...,...,...
99874,2024-12-23 12:50:00,8084.8,8088.1,8083.8,8084.8,111,90,0
99908,2024-12-23 15:40:00,8090.8,8091.6,8084.1,8084.6,225,90,0
99909,2024-12-23 15:45:00,8084.3,8086.8,8082.6,8082.8,169,90,0
99925,2024-12-23 17:05:00,8074.6,8079.6,8070.3,8070.6,555,90,0


In [80]:
prevBullish

,time,open,high,low,close,tick_volume,spread,real_volume
7,2023-07-17 19:20:00,7414.2,7415.7,7413.7,7415.7,15,200,0
20,2023-07-17 20:25:00,7416.2,7416.7,7415.5,7416.7,13,200,0
23,2023-07-17 20:40:00,7416.5,7417.2,7416.2,7417.2,9,200,0
31,2023-07-17 21:20:00,7416.7,7418.2,7416.7,7417.7,20,200,0
35,2023-07-17 21:40:00,7414.0,7414.7,7413.7,7414.7,24,200,0
...,...,...,...,...,...,...,...,...
99922,2024-12-23 16:50:00,8081.1,8088.8,8080.8,8084.3,420,90,0
99925,2024-12-23 17:05:00,8074.6,8079.6,8070.3,8070.6,555,90,0
99962,2024-12-23 20:10:00,8109.3,8113.3,8109.3,8113.3,100,100,0
99980,2024-12-23 21:40:00,8117.3,8119.3,8117.3,8118.3,82,100,0
